<a href="https://colab.research.google.com/github/Fadhliii/python-task/blob/main/mcdonald_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


AMBIL DARI GOOGLE DRIVE

In [4]:
!ls "/content/drive/My Drive/"
!ls "/content/drive/My Drive/dataset/"
!ls "/content/drive/My Drive/dataset/McDonald_s_Reviews.csv"



'Advanced Machine Learning'
'Algorithms and Complexity'
'bukti uts advanced machine learning.pdf'
'Colab Notebooks'
 dataset
'Fadhli muhamamd - 14250033 - advanced machine learning.pdf'
'Home Task: Hypothesis Testing in Python.gdoc'
 InteractiveSheet_2025-10-26_16_35_56.gsheet
 Kam23Oktober20252306.jpg
 Midterm_FadhliMuhammad_14250033.gdoc
 Muhammad_Fadhli_BasicLab.ipynb
'Semester 1'
'Technopreneurship (1).gdoc'
 Technopreneurship.gdoc
'Untitled document.gdoc'
McDonald_s_Reviews.csv
'/content/drive/My Drive/dataset/McDonald_s_Reviews.csv'


In [5]:
import pandas as pd

path = '/content/drive/My Drive/dataset/McDonald_s_Reviews.csv'

df = pd.read_csv(path, encoding='latin-1')

# cek data
df[['review', 'rating']].head()
print("Data Review McDonald's:")
display(df[['review', 'rating']].head())


Data Review McDonald's:


,review,rating
0,Why does it look like someone spit on my food?...,1 star
1,It'd McDonalds. It is what it is as far as the...,4 stars
2,Made a mobile order got to the speaker and che...,1 star
3,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,"I repeat my order 3 times in the drive thru, a...",1 star


CLEANING

In [6]:
# Fungsi untuk membersihkan rating (ambil angkanyaaja)
def clean_rating(rating_text):
    if isinstance(rating_text, str):
        return int(rating_text.split()[0]) # Ambil kata pertama ("5 stars" -> 5)
    return rating_text

# Terapkan ke kolom rating
df['rating_score'] = df['rating'].apply(clean_rating)

# Lihat sebaran datanya
print("Jumlah review per bintang:")
print(df['rating_score'].value_counts().sort_index())

Jumlah review per bintang:
rating_score
1     9431
2     3086
3     4818
4     5787
5    10274
Name: count, dtype: int64


## LABELING DULU

In [7]:
def get_sentiment(score):
    if score <= 2:
        return 'Negative'
    elif score == 3:
        return 'Neutral'
    else:
        return 'Positive'

df['sentiment'] = df['rating_score'].apply(get_sentiment)

df['sentiment'].value_counts()




,count
sentiment,
Positive,16061
Negative,12517
Neutral,4818


text processing

In [8]:


def clean_text(text):
    # Ubah jadi huruf kecil
    text = str(text).lower()
    # Hapus karakter aneh (non-ascii)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Hapus karakter spesial selain huruf dan spasi
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

# Terapkan pembersihan
df['clean_review'] = df['review'].apply(clean_text)

# Intip teks sebelum vs sesudah dibersihkan
print("SEBELUM:", df['review'].iloc[0])
print("SESUDAH:", df['clean_review'].iloc[0])

SEBELUM: Why does it look like someone spit on my food?
I had a normal transaction,  everyone was chill and polite, but now i dont want to eat this. Im trying not to think about what this milky white/clear substance is all over my food, i d*** sure am not coming back.
SESUDAH: why does it look like someone spit on my food
i had a normal transaction  everyone was chill and polite but now i dont want to eat this im trying not to think about what this milky whiteclear substance is all over my food i d sure am not coming back


In [9]:
df_final = df # pakai semua data

X = df_final['clean_review']
y = df_final['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat Pipeline:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Latih modelnya!
print("Sedang melatih model... (tunggu sebentar)")
model.fit(X_train, y_train)
print("Selesai!")

Sedang melatih model... (tunggu sebentar)
Selesai!


TEST DATA

> Add blockquote



In [11]:
# Prediksi data ujian
y_pred = model.predict(X_test)

# Tampilkan akurasi
print(f"Akurasi Model: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\nLaporan Detail:")
print(classification_report(y_test, y_pred))

print("="*30)
print("COBA TES KATA-KATA SENDIRI")
print("="*30)


kalimat_sendiri = [
    "The burger was cold and the staff was rude",    #  Negative
    "Amazing fries! Best mcdonalds ever.",           #  Positive
    "It was okay, not bad but not great either."     #  Neutral
]

hasil_prediksi = model.predict(kalimat_sendiri)

for kalimat, hasil in zip(kalimat_sendiri, hasil_prediksi):
    print(f"Review: '{kalimat}'  -->  Sentimen: {hasil}")

Akurasi Model: 79.90%

Laporan Detail:
              precision    recall  f1-score   support

    Negative       0.76      0.92      0.83      2543
     Neutral       1.00      0.19      0.33       939
    Positive       0.82      0.88      0.85      3198

    accuracy                           0.80      6680
   macro avg       0.86      0.67      0.67      6680
weighted avg       0.82      0.80      0.77      6680

COBA TES KATA-KATA SENDIRI
Review: 'The burger was cold and the staff was rude'  -->  Sentimen: Negative
Review: 'Amazing fries! Best mcdonalds ever.'  -->  Sentimen: Positive
Review: 'It was okay, not bad but not great either.'  -->  Sentimen: Positive




---



logistic
**REGRESSION**

In [12]:
from sklearn.model_selection import train_test_split

X = df["clean_review"]
y = df["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

lr_model = make_pipeline(
    TfidfVectorizer(stop_words="english", max_features=5000),
    LogisticRegression(max_iter=1000)
)

print("Training Logistic Regression...")
lr_model.fit(X_train, y_train)
print("Selesai.")


Training Logistic Regression...
Selesai.


In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = lr_model.predict(X_test)

print("Accuracy:", f"{accuracy_score(y_test, y_pred)*100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 81.08%

Classification Report:
              precision    recall  f1-score   support

    Negative       0.80      0.88      0.84      2504
     Neutral       0.69      0.31      0.43       964
    Positive       0.83      0.91      0.87      3212

    accuracy                           0.81      6680
   macro avg       0.77      0.70      0.71      6680
weighted avg       0.80      0.81      0.79      6680

Confusion Matrix:
[[2210   50  244]
 [ 319  299  346]
 [ 218   87 2907]]
